In [1]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import html5lib
from transformers import pipeline
import tensorflow as tf

2023-01-22 00:05:34.270148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 00:05:34.527694: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-22 00:05:34.546469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-22 00:05:34.546536: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
def get_source(url):
    """Return the source code of the provided url.
    
    Args: 
        url (string): URL of the page to scrape.
        
    Returns:
        response (object): HTTP response object from requests_html
    """  
    
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    
    except requests.exceptions.RequestException as e:
        print(e)

In [4]:
def scrape_google(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    response1 = get_source("https://www.google.com/search?q=" + query + '&start=10')
    
    links = list(response.html.absolute_links)
    links += list(response1.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    
    return links

In [5]:
links = scrape_google("make worklife healthier")

In [12]:
def get_body(url):
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    coverpage_data = soup1.find_all('p')
    heading1 = soup1.find_all('h1')[0].get_text()
    heading2 = soup1.find_all('h2')
    heading2 = [heading2[i].get_text() for i in range(len(heading2))]
    data = [coverpage_data[i].get_text() for i in range(len(coverpage_data))]
    
    if len(heading1) > 0:

        headings = heading1 + " ".join(heading2)
    elif len(heading1) < 1:
        headings = " ".join(heading2)
    else:
        #Run the machine learning model here ML
        pass
    final_text = " ".join(data)
    final_text = final_text.replace('.', '.<eos>')
    final_text = final_text.replace('?', '.<eos>')
    final_text = final_text.replace('!', '.<eos>')
    final_text = final_text.split('<eos>')
    return [headings, final_text]

In [13]:
def add_data_to_df(links):
    all_headings_text = []
    for i in links:
        curr = get_body(i)
        all_headings_text.append(curr)
        
    df = pd.DataFrame(all_headings_text, columns = ['Headings', 'Text'])
    return df

In [14]:
df = add_data_to_df(links)

In [15]:
df

,Headings,Text
0,25 Best Ways To Achieve Work-Life Balance and ...,[ Vantage Rewards Vantage Pulse Vantage P...
1,\n\t\t\t\t\t\t\t\t15 tips to create a healthy ...,[This is having devastating consequences on ou...
2,The Importance of Work-Life BalanceWork-Life B...,[Work-life balance has been in the news a lot ...
3,How to have a good work-life balance (hint: it...,[Products Build leaders that accelerate team p...
4,\nWork-life balance\n\nHeader menu Header menu...,[A healthy work-life balance will mean differe...
5,12 Steps to Finding Your Ideal Work-Life Balan...,"[Thanks to remote work, super-smart tech, and ..."
6,How to Improve Your Work-Life Balance TodayBal...,"[Often, work takes precedence over everything ..."
7,Try these 12 tips to improve your work-life ba...,[Products Build leaders that accelerate team p...
8,16 Best Tips To Maintain a Healthy Work-Life B...,"[Today, quality of work-life balance is one of..."
9,6 Tips For Better Work-Life Balance,[This article is part of a series on work-life...


In [17]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/home/a/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512

In [18]:
max_chunk = 500
current_chunk = 0
chunks = []



In [19]:
for sentence in df['Text'][0]:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [21]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [22]:
chunks

['\xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blog \xa0\xa0Influencers Podcast \xa0\xa0Guides & eBooks \xa0\xa0Webinars \xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blog \xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blog \xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blog \xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blog \xa0\xa0Vantage Rewards \xa0\xa0Vantage Pulse \xa0\xa0Vantage Perks \xa0\xa0Vantage Fit \xa0\xa0 Blogue Balance, a term that associates with almost everything that we do knowingly or unknowingly.  Today’s work culture revolves around the concept of balance to a great extent.  Here, the balance I’ll be talking about is the one between work and life.  Times are hectic, humans are more engaged, and achieving a perfect 

In [23]:
res = summarizer(chunks, max_length = 120, min_length=30, do_sample = False)

Token indices sequence length is longer than the specified maximum sequence length for this model (653 > 512). Running this sequence through the model will result in indexing errors
2023-01-22 11:16:57.747268: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x556ee74db730 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-01-22 11:16:57.747492: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2023-01-22 11:16:57.868418: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-22 11:16:58.280720: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [24]:
res

[{'summary_text': 'work culture revolves around the concept of balance to a great extent . achieving a perfect work-life balance seems like a far-fetched dream . balance is not only about coming to and returning from working on time .'},
 {'summary_text': "an unhealthy work-life balance disrupts the employees' physical and mental health . a healthy lifestyle is also a great stress management solution and helps the employees cope with stress smoothly ."},
 {'summary_text': "a good balance between work and life isn't possible if there's no personal time left after leaving work . a flexible work schedule helps the employees lay equal focus on their own personal and social side of life like that of their work, and it works as an alternate stress management technique . one must make sure they have a proper routine in place for the day ."},
 {'summary_text': 'remote workers are booming all over the world as millennials and Gen z’s are joining the workforce . it can be an incredibly effective